# Genetic algorithm with gym library
### In this experiment, we will try to do genetic algorithm for games: Lunar Lander and Frozen Lake. The first one is about landing with a rocket on moon using 3 engines. The second game is like a labyrinth. To do it we will use Pygad library

In [2]:
import gym
import numpy as np
import pygad

fitness function

In [3]:
def fitness_func(solution, solution_idx):
    total_reward = 0
    # Resetowanie gry
    state = env.reset(seed=42)

    for action in solution:
        state, reward, done, info , _ = env.step(action.astype(int))
        total_reward += reward

        if done:
            break

    return total_reward

In [4]:
# Tworzenie środowiska gry
env = gym.make("LunarLander-v2")

# Definicja problemu optymalizacyjnego dla algorytmu genetycznego
gene_space = [0, 1, 2, 3]
num_genes = 100
num_generations = 100
num_parents_mating = 50
sol_per_pop = 500
keep_parents = 10
parent_selection_type = "sss"
crossover_type = "single_point"
mutation_type = "random"
mutation_percent_genes = 10

# Tworzenie instancji algorytmu genetycznego
ga_instance = pygad.GA(gene_space=gene_space,
                       num_genes=num_genes,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       fitness_func=fitness_func,
                       stop_criteria="reach_105")


In [6]:
env = gym.make("LunarLander-v2")

# Rozpoczęcie optymalizacji
ga_instance.run()

# Optymalne rozwiązanie
best_solution, solution, solution_fitness= ga_instance.best_solution()

# Wydrukowanie wyników
print("Najlepsze rozwiązanie:", best_solution)
print("Wynik optymalizacji:", solution)

Najlepsze rozwiązanie: [0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 3. 0. 0. 3. 1. 1. 3. 2. 0. 1. 2. 0. 1. 0.
 3. 2. 3. 1. 1. 0. 3. 1. 1. 0. 2. 2. 3. 0. 2. 1. 0. 2. 0. 0. 0. 1. 0. 0.
 3. 0. 3. 1. 3. 2. 2. 3. 3. 3. 3. 0. 2. 1. 0. 3. 2. 1. 1. 1. 1. 2. 3. 1.
 0. 1. 2. 3. 3. 3. 3. 1. 1. 1. 3. 1. 1. 2. 1. 3. 0. 1. 2. 0. 2. 1. 1. 3.
 3. 2. 2. 3.]
Wynik optymalizacji: 116.58772144660429


In [11]:
# replay the game
env = gym.make("LunarLander-v2", render_mode="human")

state = env.reset(seed=42)
done = False
num_steps = 0

while not done and num_steps < len(best_solution):
    action = best_solution[num_steps]
    observation, reward, terminated, truncated, info = env.step(action)
    # print(action)
    env.render()

    num_steps += 1
    if terminated or truncated:
      env.close()

env.close()

AssertionError: 0.0 (<class 'numpy.float64'>) invalid 